In [ ]:
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt
import glob as glob
import os as os
import shutil as shutil
import scipy.signal as sig
from pandas.core.common import flatten



In [ ]:
assay = "Assay"
expname = "210719-Coculture-Drugs"
basepath = os.path.join(assay,expname)

In [ ]:
def makedir(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [ ]:
makedir(os.path.join(basepath,"MasksGRU"))
makedir(os.path.join(basepath,"MasksRDU"))
makedir(os.path.join(basepath,"OverlayGRU"))
makedir(os.path.join(basepath,"OverlayRDU"))
makedir(os.path.join(basepath,"GRU"))
makedir(os.path.join(basepath,"RDU"))

In [ ]:
times = ['*00h*','*12h*']

In [ ]:
def normaliseimagesgreen(img):
    clahe = cv2.createCLAHE(clipLimit=20.0,tileGridSize=(5,5))
    clahe_img = clahe.apply(img)
    clahe_img = clahe_img.astype(np.int32)
    print(np.quantile(clahe_img,0.99))

    clahe_img = ((clahe_img - 2000))
    clahe_img[clahe_img<0] = 0
    if np.amax(clahe_img) < 2500:
        nrm = cv2.convertScaleAbs(clahe_img,alpha=(255.0/10000))
    else:
        nrm = cv2.convertScaleAbs(clahe_img,alpha=(255.0/(min(np.quantile(clahe_img,0.995),50000))))#was 5000
    nrm[nrm>255]=255
    nrm[nrm<0]=0
    nrm = nrm.astype(np.uint8)
    return nrm


source = os.path.join(basepath,'RGRU')
dest = os.path.join(basepath,'GRU')
files = list(flatten([sorted(glob.glob(os.path.join(source,e))) for e in times]))
q1 = np.zeros(len(files))
q2 = np.zeros(len(files))


for i in range(len(files)):
    
    img = cv2.imread(files[i],cv2.IMREAD_UNCHANGED)
    q1[i]= np.amax(img)
    q2[i] = np.amin(img)
    img = normaliseimagesgreen(img)
    filename = os.path.join(dest,os.path.basename(files[i]))
    cv2.imwrite(filename,img)
    print(os.path.basename(files[i]))


In [ ]:
def normaliseimagesred(img,q1,i):
    
    clahe = cv2.createCLAHE(clipLimit=10.0,tileGridSize=(5,5))
    clahe_img = clahe.apply(img).astype(np.int32)
    
    q1[i] = (np.quantile(clahe_img,0.995))
    if np.quantile(clahe_img,0.995) < 50:
        nrm = cv2.convertScaleAbs(clahe_img,alpha=(255.0/1000))
        print("low")
    else:
        
        nrm = cv2.convertScaleAbs(clahe_img,alpha=(255.0/np.quantile(clahe_img,0.995)))
        #nrm = 255*(clahe_img-np.amin(clahe_img))/(np.quantile(clahe_img,0.995)-np.amin(clahe_img))
        
        
    nrm[nrm<0] = 0
    nrm[nrm>255] = 255
    
    nrm = nrm.astype(np.uint8)
    return nrm


source = os.path.join(basepath,'RRDU')
dest = os.path.join(basepath,'RDU')
files = list(flatten([sorted(glob.glob(os.path.join(source,e))) for e in times]))
q1 = np.zeros(len(files))

for i in range(len(files)):
             
    img = cv2.medianBlur(cv2.imread(files[i],cv2.IMREAD_UNCHANGED),3)
    img = normaliseimagesred(img,q1,i)
    filename = os.path.join(dest,os.path.basename(files[i]))
    cv2.imwrite(filename,img)
    print(os.path.basename(files[i]))

In [ ]:
#Run this only if the images are not adequately normalised

source = os.path.join(basepath,'RDU')
dest = os.path.join(basepath,'RDU')
files = list(flatten([sorted(glob.glob(os.path.join(source,e))) for e in times]))
q1 = np.zeros(len(files))
q2 = np.zeros(len(files))

for i in range(len(files)):
    img = cv2.imread(files[i],0)
    filename = os.path.join(dest,os.path.basename(files[i]))
    cv2.imwrite(filename,img)
    print(q1[i],q2[i],os.path.basename(files[i]))

    
source = os.path.join(basepath,'GRU')
dest = os.path.join(basepath,'GRU')
files = list(flatten([sorted(glob.glob(os.path.join(source,e))) for e in times]))
q1 = np.zeros(len(files))
q2 = np.zeros(len(files))

for i in range(len(files)):
    
    img = cv2.imread(files[i],cv2.IMREAD_UNCHANGED)
    img = img.astype(np.int16)
    
    q1[i] = np.quantile(img,0.01)
    q2[i] = np.quantile(img,1)
    if np.quantile(img,0.99) < 40:
        img = ((img)/(255))*255
        img[img<0]=0
        img[img>255]=255
        img = img.astype(np.uint8)
        print("empty")
    else:
        img = ((img-np.quantile(img,0.01))/(np.quantile(img,1)-np.quantile(img,0.01)))*255
        img[img<0]=0
        img[img>255]=255
        img = img.astype(np.uint8)

           
    
    filename = os.path.join(dest,os.path.basename(files[i]))
    cv2.imwrite(filename,img)

In [ ]:
def RemoveSmallObjects(img_file,min_size):

    nb_components, output, stats, _ = cv2.connectedComponentsWithStats(img_file, connectivity=4)
    sizes = stats[1:, -1]
    nb_components = nb_components - 1
    out = np.zeros((output.shape)).astype(np.uint8)
    for i in range(0, nb_components):
        if sizes[i] >= min_size:
            out[output == i + 1] = 255
    return out

In [ ]:
#files2 = sorted(glob.glob(os.path.join(basepath,'RDU','*B*')))

files1 = sorted(glob.glob(os.path.join(os.path.join(basepath,'GRU3'),'*')))
files2 = sorted(glob.glob(os.path.join(os.path.join(basepath,'RDU3'),'*')))

#files2 = list(flatten([sorted(glob.glob(os.path.join(basepath,'RDU',e))) for e in times]))

for i in range(len(files2)):
    print(files2[i])
    if(os.path.isfile(os.path.join(basepath,'GRU3','GRU'+os.path.basename(files2[i])[3:]))):
        gimg = cv2.imread(os.path.join(basepath,'GRU3','GRU'+os.path.basename(files2[i])[3:]),0).astype(np.uint8)
        rimg = cv2.imread(files2[i],0).astype(np.uint8)

        _,red = cv2.threshold(rimg, 40,254,cv2.THRESH_BINARY) #was 25 210707
        _,green = cv2.threshold(gimg, 60,254,cv2.THRESH_BINARY)

        red = RemoveSmallObjects(red,30)
        green = RemoveSmallObjects(green,30)

        redwrite = np.zeros((red.shape)).astype(np.uint8)
        greenwrite = np.zeros((green.shape)).astype(np.uint8)

        redwrite[np.logical_and(red > 0,green > 0) ] = 127
        redwrite[np.logical_and(red > 0,green == 0) ] = 254
        greenwrite[np.logical_and(red > 0,green > 0) ] = 127
        greenwrite[np.logical_and(red == 0,green > 0) ] = 254
        stackr = cv2.merge([red,rimg,rimg])
        stackg = cv2.merge([green,gimg,gimg])

        cv2.imwrite(os.path.join(basepath,'OverlayRDU3',os.path.basename(files2[i])),stackr)
        cv2.imwrite(os.path.join(basepath,'OverlayGRU3','GRU'+os.path.basename(files2[i])[3:]),stackg)
        cv2.imwrite(os.path.join(basepath,'MasksRDU3',os.path.basename(files2[i])),redwrite)
        cv2.imwrite(os.path.join(basepath,'MasksGRU3','GRU'+os.path.basename(files2[i])[3:]),greenwrite)

    
            
    
    

In [ ]:
files1 = sorted(glob.glob(os.path.join(os.path.join(basepath,'MasksGRU'),"*tif")))
files2 = sorted(glob.glob(os.path.join(os.path.join(basepath,'MasksRDU'),"*tif")))


outfile = open(os.path.join(basepath,expname+"FINAL"+'.csv'),"w+")
writethis = "Letter,Number,Plate,Day,Hour,Minute,Green,Red\n"
outfile.write(writethis)
for i in range(len(files1)):
    print(i)
    img = cv2.imread(files1[i],0).astype(np.uint8)
    
    img2 = cv2.imread(files2[i],0).astype(np.uint8)

    q = (os.path.join(files1[i])).split('_')
    writethis = str(q[2][0])+","+str(q[2][1:])+","+str(q[1])+","+str(q[4][0:2])+","+str(q[4][3:5])+","+str(q[4][6:8])+","+str(np.sum(img[100:1052,100:1436]/254))+","+str(np.sum(img2[100:1052,100:1436]/254))+"\n"
    outfile.write(writethis)
outfile.flush()